In [1]:
import sys
sys.path.insert(1, 
       '/Users/jeanettemumford/Dropbox/Research/Projects/RT_sims/Code')
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
from functions import *
from scipy.stats import gamma, exponnorm
import time 
from nilearn.glm.first_level.design_matrix import _cosine_drift


/usr/local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/usr/local/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [ ]:
rt_grinband_shift = gamma.rvs(a=1.7, loc=.5, scale=.49, size=10000)
shape_grinband_shift, mu_grinband_shift, sigma_grinband_shift =\
     exponnorm.fit(rt_grinband_shift*1000)
inv_lambda_grinband_shift = np.multiply(sigma_grinband_shift,
                                        shape_grinband_shift)
n_trials = 30
scan_length = 225
repetition_time = 1

mu_expnorm = mu_grinband_shift
lam_expnorm = 1 / inv_lambda_grinband_shift
sigma_expnorm = sigma_grinband_shift

max_rt = 8000
min_rt = 50
event_duration = .5
beta_scales_yes = 5
beta_scales_no = 30
nsim = 100
center_rt = True

win_sub_noise_sd_range_scales_yes = [1.2, 1.75,  3, 4, 10]
win_sub_noise_sd_range_scales_no = [.25,  .35, .5,  .75, 3]

nsim_pow = 5000
ISI_min_max_vec = [(1, 3), (2, 5), (3, 6), (4, 7)]

mu_expnorm = mu_grinband_shift
lam_expnorm = 1 / inv_lambda_grinband_shift
sigma_expnorm = sigma_grinband_shift

hp_filter = True



In [ ]:
shape_expnorm = 1 / (sigma_expnorm * lam_expnorm)
subject_specific_mu_expnorm = exponnorm.rvs(shape_expnorm, mu_expnorm,
                                            sigma_expnorm, 1) - \
                                            1 / lam_expnorm
sim_num_trials = 0
while sim_num_trials < n_trials:
    rt_trials_twice_what_needed = exponnorm.rvs(shape_expnorm,
                                            subject_specific_mu_expnorm,
                                            sigma_expnorm, n_trials * 2) 
    rt_trials_filtered = rt_trials_twice_what_needed[
        np.where((rt_trials_twice_what_needed < max_rt) &
                (rt_trials_twice_what_needed > min_rt))]
    sim_num_trials = rt_trials_filtered.shape[0]
rt_trials = rt_trials_filtered[:n_trials] / 1000
rt_trial_mean_s = np.mean(rt_trials)
ISI = np.random.uniform(low=ISI_min, high=ISI_max, size=n_trials - 1)
onsets = np.cumsum(np.append([5], rt_trials[0:(n_trials-1)]+ISI))
frame_times = np.arange(0, scan_length*repetition_time, repetition_time)
if center_rt:
    amplitudes = {'modulated': (rt_trials - rt_trial_mean_s),
                    'unmodulated': np.ones(onsets.shape)}
else:
    amplitudes = {'modulated': rt_trials,
                    'unmodulated': np.ones(onsets.shape)}

durations = {'fixed_zero': np.zeros(onsets.shape),
                'fixed_event_duration': np.zeros(onsets.shape) +
                event_duration,
                'fixed_mean_rt': np.zeros(onsets.shape) + 
            rt_trial_mean_s,
                'rt': rt_trials}

In [ ]:
# Does sim_fit_sub die if I run it by itself?
# Trying to determine if it is this function or what I do with the 
# output of this function in calc_win_sub_pow_range

nsim_check = 200000
ISI_min = 3
ISI_max = 6
sd_ind = 1
start_time = time.time()
for i in range(nsim_check):
    if i%1000 == 0:
        print(i)
        print(start_time - time.time())
        start_time = time.time()
        
    regressors = {key: {key2: {} for key2 in amplitudes} for key in durations}

    for duration_type, duration in durations.items():
        for modulation_type, amplitude in amplitudes.items():
            onsets_3col = make_3column_onsets(onsets, duration, amplitude)
            regressors[duration_type][modulation_type], _ = \
                      hemodynamic_models.compute_regressor(
                      onsets_3col, 'spm', frame_times, oversampling=10)

In [ ]:

output_unmod_beta_power_grin_hp_yes, output_rtmod_beta_power_grin_hp_yes = \
       calc_win_sub_pow_range(n_trials, scan_length, repetition_time,
                              mu_expnorm, lam_expnorm, sigma_expnorm,
                              max_rt, min_rt, event_duration, ISI_min_max_vec,
                              win_sub_noise_sd_range_scales_yes,
                              win_sub_noise_sd_range_scales_no, center_rt,
                              beta_scales_yes, beta_scales_no, hp_filter,
                              nsim_pow)

In [ ]:
output_unmod_beta_power_grin_hp_yes, output_rtmod_beta_power_grin_hp_yes = \
       calc_win_sub_pow_range2(n_trials, scan_length, repetition_time,
                              mu_expnorm, lam_expnorm, sigma_expnorm,
                              max_rt, min_rt, event_duration, ISI_min_max_vec,
                              win_sub_noise_sd_range_scales_yes,
                              win_sub_noise_sd_range_scales_no, center_rt,
                              beta_scales_yes, beta_scales_no, hp_filter,
                              nsim_pow)